In [319]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [320]:
pits_od=pd.read_excel('/Users/santiagoborgnino/Documents/Github/Smart-Tour/Datasets/pits_opendata.xls')
google_data=pd.read_csv('google_data.csv')

In [321]:
pits_od=pits_od.drop(['originalpost','categories','atencio_eq', 'phonenumber','type','tp','date','author','tags','language','pos', 'num', 'city', 'address', 'code_url', 'related_post', 'attachments', 'vignette', 'moreinfo', 'usergroup', 'post_modified', 'original_modified', 'wt', 'sigla', 'sectionname'], axis=1)

In [322]:
#Apply upper case for names
pits_od['name'] = pits_od['name'].str.upper()
pits_od['title']=pits_od['title'].str.upper()
#Remove white space at the beginning and end of the name
pits_od['name'] = pits_od['name'].str.lstrip()


In [323]:
pits_od=pits_od.drop_duplicates('id')
pits_od.head(2)

,district,gmapx,gmapy,id,name,title,excerpt,content
0,Sants-Montjuïc,41.361374,2.159711,190829.0,PARC DE MONTJUÏC,EL PARC DE MONTJUÏC,<p>El gran espai natural del parc de Montjuïc ...,<h3>Un turó amb història</h3>\r\n<p>Assentat s...
2,Sant Martí,41.411034,2.221698,190952.0,EL PARC DEL FÒRUM,EL PARC DEL FÒRUM,<p>El recinte projectat arran de mar per acoll...,<h3>Un gran esdeveniment</h3>\r\n<p>El parc de...


In [324]:
#Removing noise strings from exceprt
pits_od['excerpt'] = pits_od['excerpt'].str.replace('<p>', '')
pits_od['excerpt'] = pits_od['excerpt'].str.replace('</p', '')
pits_od['excerpt'] = pits_od['excerpt'].str.replace('.>', '')


#Removing noise strings from content
pits_od['content']=pits_od['content'].str.replace('<h3>', '' )
pits_od['content']=pits_od['content'].str.replace('</h3>', '' )
pits_od['content']=pits_od['content'].str.replace('\r\n', '' )
pits_od['content']=pits_od['content'].str.replace('<p>', '' )
pits_od['content']=pits_od['content'].str.replace('<strong>', '' )
pits_od['content']=pits_od['content'].str.replace('</strong>', '' )
pits_od['content']=pits_od['content'].str.replace('</p>', '' )
pits_od['content']=pits_od['content'].str.replace('<!-- .photo-galleria -->', '' )

print(pits_od.excerpt[0])
print(" ")
print(pits_od.content[0])

El gran espai natural del parc de Montjuïc és el millor lloc per gaudir de la natura i la cultura alhora, perquè és ple de magnífics jardins i d’instal·lacions culturals
 
Un turó amb històriaAssentat sobre el turó que recorre el barri de Sants i mira cap al mar, Montjuïc ha estat testimoni i escenari de múltiples fets transcendentals en la història de Barcelona. Es va començar a urbanitzar a partir de l’Exposició Universal del 1929. Després dels successos dramàtics de la Guerra Civil, en què el castell va funcionar com a presó, l’indret va canviar i, amb els Jocs Olímpics de 1992, es va renovar completament i va tornar a adquirir un caràcter festiu i alegre per als barcelonins.El nom del turó, de 177 metres d’alçària, ha estat un tema de controvèrsia, ja que Montjuïc en català medieval podria traduir-se com a &#8216;Mont dels jueus&#8216;, la qual cosa està avalada per l’existència d’un cementiri jueu a la muntanya.Natura, cultura i esportEn aquest gran pulmó verd de la ciutat s’hi tr

In [325]:
pits_od['excerpt'] = pits_od['excerpt'].fillna('')

In [326]:
pits_od['content']

0      Un turó amb històriaAssentat sobre el turó que...
2      Un gran esdevenimentEl parc del Fòrum és l&#82...
3      &#8220;Planeta Vida&#8221;El Museu de Ciències...
4      Natura urbanaPollancres, acàcies, magnòlies&#8...
5      Un amfiteatre naturalEntre el Castell de Montj...
                             ...                        
847    Un parc d’ociEls terrenys que ocupava la fàbri...
848    Les pintures de Pere PrunaLa pintura de Pruna ...
849    Espai de prestigiPensat per ser un recinte des...
850    Activitats al carrerL’eix, que forma part de l...
851                                                  NaN
Name: content, Length: 851, dtype: object

In [327]:
google_data.set_index('name', inplace=True)
pits_od.set_index('title', inplace=True)

In [328]:
pits_od=pits_od.join(google_data, how='inner')

In [329]:
pits_od.reset_index(inplace=True)

In [330]:
pits_od.shape

(52, 40)

In [331]:
pits_od.columns

Index(['index', 'district', 'gmapx', 'gmapy', 'id', 'name', 'excerpt',
       'content', 'categoryName', 'temporarilyClosed', 'permanentlyClosed',
       'totalScore', 'location', 'popularTimesHistogram', 'reviewsCount',
       'latitud', 'longitud', 'tipo', '1994', '2000', '2011', '2014', '2015',
       '2016', '2017', '2018', '2019', 'Average', '1', '2', '3', '4', '5', '6',
       '7', '8', '9', '10', '11', '12'],
      dtype='object')

In [332]:
pits_od=pits_od[~pits_od['excerpt'].isna()]

In [333]:
pits_od['excerpt']=pits_od['excerpt']+pits_od['excerpt']

In [334]:
tfidf = TfidfVectorizer(max_df=0.95)
tfidf_matrix = tfidf.fit_transform(pits_od['excerpt'])

In [335]:
print(tfidf.get_feature_names())

['14', '14la', '1888', '1893', '1926', '1929', '1946', '1991', '2001', '2001l', '450', 'abraça', 'accedeix', 'accedir', 'acomiada', 'actiu', 'activitats', 'actualitat', 'actualitatnascuda', 'actualment', 'acull', 'agosarades', 'aixecar', 'així', 'al', 'alberga', 'allotja', 'als', 'alta', 'altres', 'alça', 'amaga', 'amagat', 'amb', 'animals', 'antenes', 'antic', 'antiga', 'antigues', 'antoni', 'any', 'anys', 'apropa', 'apropar', 'aquest', 'aquàrium', 'ara', 'arqueologia', 'arqueològiques', 'arquitecte', 'arquitectura', 'arquitectònic', 'arreu', 'art', 'arts', 'artística', 'artístiques', 'atalaia', 'atraccions', 'autòmats', 'avantguardista', 'avinguda', 'avui', 'banc', 'barcelona', 'barcelonael', 'barcelonal', 'barcelonala', 'barcelonaobra', 'barcelonasímbol', 'barcelonatestimoni', 'barri', 'barroc', 'barça', 'batlló', 'bell', 'bellesa', 'blau', 'born', 'cabines', 'cada', 'caixaforum', 'camp', 'can', 'cantonada', 'captiva', 'característica', 'carrers', 'casa', 'castell', 'catalunya', 'ca

In [336]:
tfidf_matrix

<52x583 sparse matrix of type '<class 'numpy.float64'>'
	with 1178 stored elements in Compressed Sparse Row format>

In [337]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [338]:
cosine_sim[0:4]

array([[1.        , 0.0477531 , 0.16166871, 0.09571576, 0.00899099,
        0.01610952, 0.01715695, 0.01916097, 0.04140001, 0.07704848,
        0.03467897, 0.03262236, 0.03166424, 0.11421856, 0.03575805,
        0.01609321, 0.05873163, 0.10107527, 0.0891486 , 0.03406341,
        0.05568011, 0.05098215, 0.02627468, 0.03755417, 0.01403523,
        0.02028829, 0.06158919, 0.13595971, 0.02437658, 0.03653691,
        0.        , 0.12878259, 0.03919758, 0.04485079, 0.1504129 ,
        0.04523128, 0.02322731, 0.06152428, 0.02807903, 0.0347144 ,
        0.09606183, 0.08751909, 0.05383102, 0.1017672 , 0.0208968 ,
        0.0588491 , 0.02791175, 0.04984414, 0.02376657, 0.03635661,
        0.0314586 , 0.03860105],
       [0.0477531 , 1.        , 0.05992844, 0.12688462, 0.07248643,
        0.03933579, 0.09324398, 0.07029934, 0.20065025, 0.0284651 ,
        0.08106242, 0.13288364, 0.05498135, 0.14802331, 0.13363273,
        0.04793501, 0.14812484, 0.20647178, 0.09478886, 0.12048637,
        0.07030

In [339]:
indices = pd.Series(pits_od.index, index=pits_od['index'])

In [340]:
indices

index
AQUARIUM BARCELONA DE GREG                            0
BARCELONA MUSEUM OF CONTEMPORARY ART                  1
BARCELONA ZOO                                         2
CAIXAFORUM BARCELONA                                  3
CASA BATLLÓ                                           4
CASA MILÀ                                             5
CASA VICENS GAUDÍ                                     6
CATALAN MUSEUM OF ARCHAEOLOGY                         7
CENTRE DE CULTURA CONTEMPORÀNIA DE BARCELONA          8
COLUMBUS MONUMENT                                     9
COSMOCAIXA BARCELONA                                 10
EL BORN CENTRE DE CULTURA I MEMÒRIA                  11
FABRA I COATS                                        12
FC BARCELONA MUSEUM                                  13
FUNDACIÓ ANTONI TÀPIES                               14
FUNDACIÓ FOTO COLECTANIA                             15
FUNDACIÓ SUÑOL                                       16
FUNDACIÓ VILA CASAS                       

In [341]:
indices = indices[~indices.index.duplicated(keep='last')]

In [342]:
def content_based_recommender(title, cosine_sim, dataframe):

    dataframe = dataframe[~dataframe["index"].isna()]
    indices = pd.Series(dataframe.index, index=dataframe['index'])
    indices = indices[~indices.index.duplicated(keep='last')]

    movie_index = indices[title]

    similarity_scores = pd.DataFrame(cosine_sim[movie_index], columns=["score"])

    movie_indices = similarity_scores.sort_values("score", ascending=False)[1:11].index
    return dataframe['index'].iloc[movie_indices]

In [349]:
content_based_recommender("LA SAGRADA FAMILIA", cosine_sim, pits_od)

51                VIRREINA PALACE
5                       CASA MILÀ
33    MUSEU DEL MODERNISME CATALÀ
40      PARC DEL LABERINT D'HORTA
15       FUNDACIÓ FOTO COLECTANIA
18             GAUDÍ HOUSE MUSEUM
50            TORRE DE COLLSEROLA
26                MONTJUÏC CASTLE
24     MIES VAN DER ROHE PAVILION
41                     PARK GÜELL
Name: index, dtype: object

In [344]:
pits_od[pits_od['name']=='MUSEU DE CIÈNCIES NATURALS DE BARCELONA']

,index,district,gmapx,gmapy,id,name,excerpt,content,categoryName,temporarilyClosed,...,3,4,5,6,7,8,9,10,11,12
30,MUSEU DE CIÈNCIES NATURALS DE BARCELONA,Sant Martí,41.41078,2.220482,190945.0,MUSEU DE CIÈNCIES NATURALS DE BARCELONA,"Ubicat en el singular Edifici Fòrum, el Museu ...",&#8220;Planeta Vida&#8221;El Museu de Ciències...,Natural history museum,False,...,15255.077417,20867.947588,23557.568628,26338.719198,30748.810596,30751.662672,25133.631203,20205.384706,12791.534606,11630.69347


In [348]:
pits_od[pits_od['index'].str.contains('SAGRADA')]

,index,district,gmapx,gmapy,id,name,excerpt,content,categoryName,temporarilyClosed,...,3,4,5,6,7,8,9,10,11,12
21,LA SAGRADA FAMILIA,Eixample,41.403942,2.17497,190827.0,TEMPLE EXPIATORI DE LA SAGRADA FAMÍLIA - BASÍLICA,"Elevant-se al cel, la Sagrada Família és la si...",Un prodigi modernistaEl temple de la Sagrada F...,Basilica,True,...,293552.64084,401560.802076,453317.036262,506834.568294,591697.721776,591752.604135,483645.124311,388811.139899,246146.817957,223808.813911
